### Aprenidzaje No supervizado (Solo variables independientes)
es un tipo de ap´rendizaje no supervizado, que se utiliza cuando tiiee datos no etiquetados es decir datos sin categorias o grupos definidos es decir datos sin categoria o grupos definidos
El objetivo de ewste algoritmo es encontrar grupos en los datos.
los puntos de datos se agrupan segun la similitud de caracteristicas 
las utilidades de este tipo de aprendizaje se dan por ejemplo:
celulas cancerosa, buscar pallabras , identificar valores atipicos en el comportamiento

En este caso se esta aplicando a la data de en la cuales  tiene como variables

en este caso tomamos solo los valores independientes como son la latitud , longitud fecha , hora ,numero depasajeros, dia semana 

###### solo variables independientes



In [1]:
import findspark
#findspark.init("/usr/local/spark/spark-3.1.1-bin-hadoop2.7")    #para linux
findspark.init() 

In [2]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col
# instantiate spark environment
sc = SparkContext("local[*]", "Decision Tree Classifier")
spark = SparkSession(sc)
# load the dataset
df = spark.read.csv("newDatapandas.csv", inferSchema=True, header=True)
df.show(5)

+-----------+--------------------+-----------------+-----------------+------------------+------------------+---------------+--------------------+--------------------+---------+----------+----+
|fare_amount|     pickup_datetime| pickup_longitude|  pickup_latitude| dropoff_longitude|  dropoff_latitude|passenger_count|        dif_latitude|       dif_longitude|distancia|dia_semana|hora|
+-----------+--------------------+-----------------+-----------------+------------------+------------------+---------------+--------------------+--------------------+---------+----------+----+
|        8.0|2015-03-12 23:14:...|-73.9931411743164|40.72793960571289|-73.99661254882812|40.744529724121094|              2|0.016590118408203125| 0.00347137451171875|1.8683056|         4|  23|
|       10.0|2013-08-21 08:38:...|       -73.964837|        40.769933|        -73.983462|         40.761655|              1|0.008278000000004226|0.018625000000000114|1.8191968|         3|   8|
|        5.5|2014-01-23 18:40:...| 

In [3]:
df.columns

['fare_amount',
 'pickup_datetime',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude',
 'passenger_count',
 'dif_latitude',
 'dif_longitude',
 'distancia',
 'dia_semana',
 'hora']

In [4]:
from pyspark.ml.linalg import Vectors

from pyspark.ml.feature import VectorAssembler
columns = ['pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude',
 'passenger_count',
 'dia_semana',
 'hora']

In [5]:
#lo que hacemos es tomar un conjunto de columnas y definir una caracteristuca en particular pra nuestras caracteristuicas

vect_assembler = VectorAssembler(inputCols = columns, outputCol ='features')


In [6]:
final_data = vect_assembler.transform(df)

la estandarizacion del metodo de tranformacion del conjunto de datos es unm requisito comun para

In [8]:
from pyspark.ml.feature import StandardScaler

In [9]:
scaler = StandardScaler(inputCol ="features", outputCol ="scaledFeatures", withMean=False, withStd=True)

In [10]:
scalerModel = scaler.fit(final_data)

In [11]:
cluster_final_data = scalerModel.transform(final_data)

In [12]:
from pyspark.ml.clustering import KMeans

In [13]:
kmeans3 = KMeans(featuresCol ='scaledFeatures', k =3)
kmeans2 = KMeans(featuresCol ='scaledFeatures', k =2)

In [14]:
model_k3 =kmeans3.fit(cluster_final_data)
model_k2 =kmeans2.fit(cluster_final_data)

In [15]:
predictions3 = model_k3.transform(cluster_final_data)
predictions2 = model_k2.transform(cluster_final_data)

In [17]:

from pyspark.ml.evaluation import ClusteringEvaluator
evaluator = ClusteringEvaluator()

In [18]:
silhouette_3 = evaluator.evaluate(predictions3)
silhouette_2 = evaluator.evaluate(predictions2)

In [19]:
print("Con k =3")
print("Suuma de los errores al cuadrado = " + str(silhouette_3))
print("Con k =2")
print("Suuma de los errores al cuadrado = " + str(silhouette_2))

Con k =3
Suuma de los errores al cuadrado = -0.014664806474756986
Con k =2
Suuma de los errores al cuadrado = 0.9593956831344325


In [20]:
for k in range(2,9):
    kmeans  = KMeans(featuresCol ='scaledFeatures', k =k)
    model =kmeans.fit(cluster_final_data)
    predictions = model.transform(cluster_final_data)
    silhouette = evaluator.evaluate(predictions)
    print("Con k ={}".format(k))
    print("Suuma de los errores al cuadrado = " + str(silhouette))

Con k =2
Suuma de los errores al cuadrado = 0.9593956831344325
Con k =3
Suuma de los errores al cuadrado = -0.014664806474756986
Con k =4
Suuma de los errores al cuadrado = -0.5592767797892542
Con k =5
Suuma de los errores al cuadrado = -0.016216138812811413
Con k =6
Suuma de los errores al cuadrado = -0.06916233321257086
Con k =7
Suuma de los errores al cuadrado = -0.07510872544253545
Con k =8
Suuma de los errores al cuadrado = -0.16805817365567766


In [21]:
model_k3.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-------+
|prediction|  count|
+----------+-------+
|         1|  45295|
|         2|1074777|
|         0|1099101|
+----------+-------+



In [22]:
model_k2.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-------+
|prediction|  count|
+----------+-------+
|         1|  45295|
|         0|2173878|
+----------+-------+

